In [ ]:
%matplotlib inline
%matplotlib notebook
import csv
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from random import randint
import motion_tracking2 as motion_tracking
from scipy.optimize import linear_sum_assignment
import copy
import math
import traj_dist.distance as tdist
import itertools
import scipy.signal
import numpy.linalg
import pylab as py
from collections import defaultdict
import xml.etree.ElementTree as ET

In [ ]:
FONT = cv2.FONT_HERSHEY_SIMPLEX
EPS = 1e-3 # float underflow tolerance
PERSPECTIVE_TRANSFORM = np.array([
        [1.51453582e+00, 5.20852351e+00, 6.64802632e+01],
        [7.96413100e-02, 4.86708608e+00, 1.12297242e+01],
        [4.72112167e-04, 2.16583992e-03, 6.11895338e-01]
    ])

## Ground Truths

In [ ]:
# frame_id corresponds to index of list
# each list element is dict: key=object_iD, val=(x,y,w,h)
# http://www.milanton.de/data/
def get_ground_truths(is_center=False):
    tree = ET.parse('PETS2009-S2L1.xml')
    ground_truths = []
    cur_frame = None
    cur_object = None
    root = tree.getroot()
    for element in root.iter():  # hardcoded XML ordering
        #print("%s - %s" % (element.tag, element.attrib))
        if element.tag == 'frame':
            ground_truths.append({})
            cur_frame = int(element.attrib['number'])

        elif element.tag == 'object':
            cur_object = int(element.attrib['id'])

        elif element.tag == 'box':
            xc, yc = float(element.attrib['xc']), float(element.attrib['yc'])
            w, h   = float(element.attrib['w']), float(element.attrib['h']) 
            x, y   = xc-w/2, yc-h/2
            if is_center:
                bbox = (xc,yc,w,h)
            else:
                bbox = (x,y,w,h)
            ground_truths[cur_frame][cur_object] = bbox
    return ground_truths

ground_truth = get_ground_truths()

## Image Caching

In [ ]:
sequence_path = "sequence"
background_img = "background.png"

# path = base path
# folders = folders within path that contain images
# Returns the opened images for every image in path/ within every folder in folders
def get_files(path):
    file_list = list()
    for (dir_path, dir_names, file_names) in os.walk(path):
        file_list += [cv2.imread(os.path.join(dir_path, file)) for file in file_names]
    return file_list

In [ ]:
images = get_files(sequence_path)
bg_img = cv2.imread(background_img)
bg_img_gray = cv2.cvtColor(bg_img, cv2.COLOR_BGR2GRAY)

# Background Subtraction

In [ ]:
k_open = np.ones((3,3),np.uint8)
k_close = np.ones((15,15), np.uint8)
k_erode = np.ones((5,5),np.uint8)
k_dilate = np.ones((5,5),np.uint8)

def bg_subtract(img): # uses globals
    #img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    subtract_1 = cv2.absdiff(bg_img, img)
    
    subtract = np.max(subtract_1, axis=2)
    
    ret, thresholded = cv2.threshold(subtract, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    morphed = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, k_open)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_ERODE, k_erode)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_DILATE, k_dilate)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_CLOSE, k_close)
    return morphed

# Pedestrian Detection

# YOLO Detector Code

In [ ]:
class YoloDetector:
    def __init__(self, frame_width, frame_height):
        self.width = frame_width
        self.height = frame_height
        self.net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        layer_names = self.net.getLayerNames()
        self.output_layers = [layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]
        
        #self.colors = np.random.uniform(0, 255, size=(len(classes), 3))
        #self.font = cv2.FONT_HERSHEY_PLAIN
        
    def get_detection(self, img):
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * self.width)
                    center_y = int(detection[1] * self.height)
                    w = int(detection[2] * self.width)
                    h = int(detection[3] * self.height)
                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    
                    # Retain only pedestrians
                    label = str(self.classes[class_id])
                    if label == 'person':
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
        
        # Remove overlapping duplicate boxes
        best_boxes = []
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        for index_array in indexes:
            best_boxes.append(boxes[index_array[0]])
        return best_boxes
        #return boxes, indexes, class_ids


def draw_rectangle(img, bbox, label, font, color,thickness=2):
    cur_x, cur_y, cur_w, cur_h = bbox
    img = cv2.rectangle(img, (cur_x, cur_y), (cur_x + cur_w, cur_y + cur_h), color, thickness)
    if label != None:
        img = cv2.putText(img, str(label), (cur_x+int(cur_w/2),cur_y-2), font, 0.5, color,thickness)
    return img

def draw_rectangles(boxes, img, color):
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        #cv2.putText(img, label, (x, y + 30), font, 3, (255,0,0), 3)
    return img

def centroid(bbox):
    x,y,w,h = bbox[:4]
    return (x + int(w/2), y + int(h/2))

def centroid_list(bboxes):
    centroids = []
    for i in range(len(bboxes)):
        x,y,w,h = bboxes[i]
        centroids.append((x + int(w/2), y + int(h/2)))
    return centroids

# Pedestrian Tracking

In [ ]:
def calc_histogram_feature(img, mask, bbox, is_norm): # color histogram
    # if no bounded img is grabbed; bbox is no longer valid; insert black image
    if motion_tracking.invalid_bbox(img,bbox):
        bounded_img = np.zeros((21,21,3), dtype=np.uint8)
        bounded_mask = np.zeros((21,21), dtype=np.uint8)
        thres1, thres2 = 7, 14
    else:
        x,y,w,h = bbox[:4]
        thres1, thres2 = int(h/3), int(h*2/3)
        bounded_img = img[y:y+h, x:x+w]
        bounded_mask = mask[y:y+h, x:x+w]
    
    hsv_img = cv2.cvtColor(bounded_img,cv2.COLOR_BGR2HSV)
    if type(hsv_img) == type(None): # debug only
        print(bounded_img.size, bbox, motion_tracking.clip_ords(img,bbox))
    hsv_parts = [ hsv_img[:,:thres1], hsv_img[:,thres1:thres2], hsv_img[:,thres2:] ]
    mask_parts = [ bounded_mask[:,:thres1], bounded_mask[:,thres1:thres2], bounded_mask[:,thres2:] ]
    hist_parts = []
    hist_contribution = np.zeros(len(hsv_parts))
    
    for i in range(len(hsv_parts)):
        particle_subhist = cv2.calcHist([hsv_parts[i]], [0, 1], mask_parts[i], [180, 256], [0, 180, 0, 256])
        particle_subhist_sum = particle_subhist.sum()
        if is_norm and particle_subhist_sum > 0: # normalise by number of counted occurrences
            particle_subhist /= particle_subhist_sum
        hist_parts.append(particle_subhist)
        hist_contribution[i] = particle_subhist_sum
    return hist_parts, hist_contribution
    # histogram(s) of img split as three columns; num masked entries in each subhist

def random_color():
    bright = [np.random.randint(215,256)] # note: one above the largest (signed) integer
    dark = [np.random.randint(0,71)]
    overall = [np.random.randint(0,256)] + bright + dark
    np.random.shuffle(overall)
    return tuple(map(lambda i: int(i), overall))

def generic_text(img, text, loc):
    img = cv2.putText(img, text, loc, FONT, 0.7, (0,0,0),4)
    img = cv2.putText(img, text, loc, FONT, 0.7, (170,0,255),2)
    return img
    
class Person:
    # prediction params (constant velocity)
    VELOCITY_WINDOW_FRAMES = 10
    # tracking threshold params
    INNER_BORDER_RECT = (398, 177, 67, 187) # (x,y,w,h) ROI rect between inner (lamppost) and middle regions
    BORDER_PX = 20 # edge between middle and outer regions
    CONSEC_TRACKED_FRAMES_THRESHOLD_INNER = 50
    CONSEC_TRACKED_FRAMES_THRESHOLD_MIDDLE = 10
    CONSEC_TRACKED_FRAMES_THRESHOLD_OUTER = 5
    SUDDEN_FP_THRES = 10 # frames
    # task-specific
    TRAJECTORY_DRAW_LENGTH = 100
    
    def __init__(self, bbox, frame_width, frame_height, num_particles, person_id, img, mask, hc):
        x,y,w,h,__,__ = bbox
        self.person_id = person_id
        self.bbox_history = [] # item: (x,y,w,h,frameID,isTracked)
        self.consec_frames_tracked = 0
        self.velocity_history = [] # item: (vx,vy,std,frameID)
        self.color = random_color()
        
        #self.tracker = motion_tracking.ParticleFilter(frame_width, frame_height, num_particles)
        self.tracker = motion_tracking.ParticleFilter(w, h, num_particles, x, y)
        self.last_prediction = None # just: (x,y,w,h)
        self.prev_normhist, self.prev_normhist_nentries = calc_histogram_feature(img, mask, bbox, True)
        self.update_tracker(bbox, img, mask, hc, None, None, False)
    
    def get_latest_ords(self):
        return self.bbox_history[-1]

    def predict_tracker(self, vx, vy, std, img, frame_id):
        self.velocity_history.append((vx,vy,std,frame_id))
        max_index = len(self.velocity_history)-1
        avg_vx, avg_vy, count = 0, 0, 0
        for i in range(max_index, max(-1,max_index-self.VELOCITY_WINDOW_FRAMES), -1):
            avg_vx += self.velocity_history[i][0]
            avg_vy += self.velocity_history[i][1]
            count += 1
        avg_vx /= count
        avg_vy /= count
        #print(f'{frame_id}: p{self.person_id}, avg_vx={round(avg_vx,3)}, avg_vy={round(avg_vy,3)}')
        self.tracker.predict(avg_vx,avg_vy,std) # shift the (unweighted) particles
        ux, uy, __, __ = self.tracker.estimate() # weighted avg of particles
        x,y,w,h,frame_id, is_tracked = self.get_latest_ords()
        predicted_bbox = motion_tracking.clip_ords(img, (ux, uy, w, h))
        self.last_prediction = predicted_bbox
        return predicted_bbox
    
    def update_tracker(self,bbox, img,mask,hc, std_m,alpha, is_hist_method):
        # add to bbox path history and miscell counters
        x,y,w,h = motion_tracking.clip_ords(img, bbox)
        __,__,__,__,frame_id, is_tracked = bbox
        bbox = (x,y,w,h,frame_id,is_tracked) # clipped
        self.bbox_history.append(bbox)
        if is_tracked:
            self.consec_frames_tracked += 1
        else:
            self.consec_frames_tracked = 0
        
        # interpolate with new histogram (of identical bins)
        cur_hist, cur_normhist_nentries = calc_histogram_feature(img, mask, bbox, True)
        for i in range(len(cur_hist)):
            self.prev_normhist[i] = (1-hc) * cur_hist[i] + hc * self.prev_normhist[i]
            self.prev_normhist[i][ self.prev_normhist[i] < EPS ] = 0
            self.prev_normhist_nentries[i] = (1-hc) * cur_normhist_nentries[i] + hc * self.prev_normhist_nentries[i]
        
        # (computes the weights for the particles; does not do other book-keeping/side-effects)
        if is_hist_method:
            # update weights (from histogram)
            wd = self.tracker.get_particle_detection_weights(x,y,std_m)
            wc = self.tracker.get_color_appearance_weights(img,mask,w,h, self.prev_normhist,
                                                           self.prev_normhist_nentries, 
                                                           calc_histogram_feature)
            self.tracker.update_dual_weights(wd, wc, alpha)
        else:
            self.tracker.update(x,y) # update weights (from Euclidean)
        self.tracker.resample(method='residual') # new set of particles
    
    def is_stale_tracker(self, img): # True --> stale
        cx, cy, cw, ch, __, is_tracked = self.get_latest_ords()
        cbbox = (cx,cy,cw,ch)
        img_h, img_w = img.shape[:2]
        # check if tracker is currently associated to a detection
        if not is_tracked:
            return False
        # check if tracker bounds are invalid
        if motion_tracking.invalid_bbox(img, cbbox):
            return True
        
        # check if last detection occurred a while ago
        # (threshold will be lower near edge of image)
        consec_tracked_frames_threshold = self.CONSEC_TRACKED_FRAMES_THRESHOLD_MIDDLE
        # ** inner-middle boundary (basically hardcoded)
        pole_x0, pole_y0, pole_w, pole_h = self.INNER_BORDER_RECT
        pole_x1, pole_y1 = pole_x0+pole_w, pole_y0+pole_h
        if not ((cx+cw) < pole_x0 or (cy+ch) < pole_y0 or cx > pole_x1 or cy > pole_y1):
            # not completely outside the inner boundary, so qualify as being inside
            consec_tracked_frames_threshold = self.CONSEC_TRACKED_FRAMES_THRESHOLD_INNER
        else: # (elif)
            # ** middle-outer boundary
            by0,by1 = self.BORDER_PX, img_h-self.BORDER_PX
            bx0,bx1 = self.BORDER_PX, img_w-self.BORDER_PX
            if cx < bx0 or cy < by0 or (cx+cw) > bx1 or (cy+ch) > by1:
                # bbox is on the edge of the image
                consec_tracked_frames_threshold = self.CONSEC_TRACKED_FRAMES_THRESHOLD_OUTER
            
        if self.consec_frames_tracked >= consec_tracked_frames_threshold:
            return True
        
        # check if tracker was newly created on a false positive that has now disappeared
        if len(self.bbox_history) <= self.SUDDEN_FP_THRES: 
            is_tracked_list = [b[-1] for i,b in enumerate(self.bbox_history) if i < self.SUDDEN_FP_THRES]
            if is_tracked_list.count(True) > int(self.SUDDEN_FP_THRES/2):
                return True # short early path, with majority of bboxes being tracked
        return False # tracker not stale
    
    def __str__(self):
        bbox = None
        if len(self.bbox_history) > 0:
            bbox = self.get_latest_ords()
        return f'Person({bbox})'
    
    def draw(self, img, color=None, enable_particles=False, draw_last_predicted=False): # global FONT
        if color == None:
            color = self.color
        if draw_last_predicted:
            x,y,w,h = self.last_prediction
        else:
            x,y,w,h,__, __ = self.get_latest_ords()
        img = cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
        if enable_particles:
            self.tracker.drawParticles(img, color, radius=1)
        img = cv2.putText(img, str(self.person_id), (x+int(w/2),y-2), FONT, 0.5, color,2)
        return img
        
    def draw_trajectory(self, img, color=None):
        if color == None:
            color = self.color
        if len(self.bbox_history) <= 0:
            return img
        # draw backwards (from most recent location)
        prev_center = centroid(self.get_latest_ords())
        img = cv2.circle(img, prev_center, 1, color)
        if len(self.bbox_history) <= 1:
            return img
        for count, i in enumerate(range(len(self.bbox_history)-1,0,-1)):
            cur_center = centroid(self.bbox_history[i])
            img = cv2.line(img, prev_center, cur_center, color, thickness=2)
            prev_center = cur_center
            if count >= self.TRAJECTORY_DRAW_LENGTH:
                break
        return img
        
def get_optical_flow(prev_img_gray, cur_img_gray):
    height,width = cur_img_gray.shape
    hsv = np.zeros((height,width,3), dtype=cur_img_gray.dtype)
    hsv[...,1] = 255
    # optical flow (direction and velocity)
    flow = cv2.calcOpticalFlowFarneback(prev_img_gray,cur_img_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    return mag, ang, rgb, flow

# https://stackoverflow.com/questions/27644388/optical-flow-using-opencv-horizontal-and-vertical-components
def draw_flow(img, flow, step=16): # flow lines in direction of motion
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

def get_values_from_mask(array_np, mask):
    array_np = array_np.ravel()
    mask = mask.ravel()
    new_array = []
    for i in range(mask.size):
        if mask[i] > 0 or mask[i] == True:
            new_array.append(array_np[i])
    if len(new_array) == 0:
        new_array = [0]
    return new_array

def get_velocity_stats(p, flow_res, mask=None): # tracker (for bbox); optical flow
    # get velocity, but using box from prev frame
    prev_bbox = p.get_latest_ords()
    px,py,pw,ph, prev_n, is_tracked = prev_bbox
    # use only the 'torso'
    px = px + int(pw/4)
    py = py + int(ph/4)
    pw = int(pw/2)
    ph = int(ph/4)
    p_velocities = flow_res[py:(py+ph), px:(px+pw), :]
    if p_velocities.size == 0:
        return 0,0,0
    x_velocities = p_velocities[:,:,0]
    y_velocities = p_velocities[:,:,1]
    if type(mask) == type(None):
        cur_vx = np.average(x_velocities)
        cur_vy = np.average(y_velocities)
    else:
        bounded_mask = mask[py:(py+ph), px:(px+pw)]
        cur_vx = np.average(np.array(get_values_from_mask(x_velocities, bounded_mask)))
        cur_vy = np.average(np.array(get_values_from_mask(y_velocities, bounded_mask)))
    cur_std = np.std(p_velocities)
    return cur_vx, cur_vy, cur_std

#####[UNUSED]#############################
# https://stackoverflow.com/questions/27152904/calculate-overlapped-area-between-two-rectangles
def area_overlap(ref_bbox, target_bbox):
    rx, ry, rw, rh = ref_bbox
    tx, ty, tw, th = target_bbox
    dx = min(rx+rw, tx+tw) - max(rx,tx)
    dy = min(ry+rh, ty+th) - max(ry,ty)
    if dx > 0 and dy > 0:
        return dx * dy
    return 0

def ratio_overlap(ref_bbox, target_bbox): # should replace with IOU calc instead
    rx, ry, rw, rh = ref_bbox
    return area_overlap(ref_bbox, target_bbox) / (rw*rh)

# assumes detection is flawless and generates only 1 bbox per pedestrian
# usage: untracked_index = get_index_max_overlap(predicted_bbox, untracked_boxes, OVERLAP_THRESHOLD_RATIO)
def get_index_max_overlap(ref_bbox, bbox_list, ratio_threshold):
    if len(bbox_list) == 0:
        return None
    best_index = 0
    best_area = area_overlap(ref_bbox, bbox_list[best_index])
    for i in range(1,len(bbox_list)):
        cur_area = area_overlap(ref_bbox, bbox_list[i])
        if cur_area > best_area:
            best_index = i
            best_area = cur_area
    # compare ratio (also filters out results with area = 0)
    x,y,w,h = ref_bbox
    best_ratio = best_area / (w*h)
    if best_ratio >= ratio_threshold:
        return best_index
    return None
#####[END OF UNUSED]######################


# https://stackoverflow.com/questions/26363257/tracking-multiple-moving-objects-with-kalmanfilter-in-opencv-c-how-to-assign
# http://www.hungarianalgorithm.com/examplehungarianalgorithm.php
# https://stackoverflow.com/questions/28050678/algorithm-for-matching-point-sets
def hungarian_algorithm(cur_people, detected_boxes, euclidean_thres):
    # set up matrix (in same order as lists)
    # ** cost = Euclidean distance
    # ** row = trackers; col = detected boxes
    matrix = np.zeros((len(cur_people), len(detected_boxes)))
    for r in range(len(cur_people)):
        px, py, pw, ph, __, __ = cur_people[r].get_latest_ords()
        pxc, pyc = px+int(pw/2), py+int(ph/2)
        for c in range(len(detected_boxes)):
            # compute cost using centroid
            bx, by, bw, bh = detected_boxes[c]
            bxc, byc = bx+int(bw/2), by+int(bh/2)
            matrix[r,c] = np.sqrt((pxc-bxc)**2 + (pyc-byc)**2)
    # run min-cost (returns corresponding (r,c) pairs)
    paired_tracker_idx, paired_detected_idx = linear_sum_assignment(matrix)
    
    # disassociate pairs that are physically too far away
    idx_to_delete = []
    for i in range(len(paired_tracker_idx)):
        r = paired_tracker_idx[i]
        c = paired_detected_idx[i]
        if matrix[r,c] > euclidean_thres:
            idx_to_delete.append(i)
    paired_tracker_idx = np.delete(paired_tracker_idx, idx_to_delete)
    paired_detected_idx = np.delete(paired_detected_idx, idx_to_delete)
    
    # find all unpaired trackers / detected boxes
    unpaired_tracker_idx = np.setdiff1d(range(len(cur_people)), paired_tracker_idx)
    unpaired_detected_idx = np.setdiff1d(range(len(detected_boxes)), paired_detected_idx)
    return paired_tracker_idx, paired_detected_idx, unpaired_tracker_idx, unpaired_detected_idx

# Group Object

In [ ]:
class Group:
    def __init__(self, people, gID):
        # Keep list of people within the group
        self.people = people
        # Unique group ID for interframe association, and group dynamics analysis
        self.gID = gID
        # Store unique group colour for display purposes
        self.color = random_color()
    
    def get_group_centroid(self):
        cent = (0,0)
        for i, p in enumerate(self.people):
            p_cent = centroid(p.get_latest_ords()[:4])
            c_x = cent[0] + p_cent[0]
            c_y = cent[1] + p_cent[1]
            cent = (c_x, c_y)
        return (int(cent[0]/len(self.people)),int(cent[1]/len(self.people)))

    # Returns average magnitude and angle of velocity over all people within group
    def get_group_velocity(self):
        mag = 0
        ang = 0
        for i, p in enumerate(self.people):
            if len(p.bbox_history) > 1:
                p_mag, p_ang = velocity(p.bbox_history[-2], p.bbox_history[-1], 1)
            else:
                p_mag, p_ang = 0, 0
            mag = mag + p_mag
            ang = ang + p_ang
        return mag/len(self.people), ang/len(self.people)
    
#     def get_group_velocity(self):
#         v = (0,0)
#         for i, p in enumerate(self.people):
#             if len(p.velocity_history) > 0:
#                 vx, vy = velocity_components(p.bbox_history[-2], p.bbox_history[-1], 1)
#             else:
#                 vx, vy = 0, 0
#             v = (v[0] + vx, v[1] + vy)
#         return (v[0]/len(self.people), v[1]/len(self.people))

In [ ]:
# Takes in an array of nx4 points, each entry of (cx, cy, vx, vy) corresponds to a single pedestrian
# Outputs a pairwise euclidean centroid distance matrix, velocity distance matrix, and a list of IDs [for indexing pedestrian list]
def dist_mats(pts):
    d_c = np.zeros((len(pts),len(pts)))
    d_v = np.zeros((len(pts),len(pts)))
    ids = []
    for p1, _ in enumerate(pts):
#         ids.append(people[p1].person_id)
        ids.append(p1)
        for p2, _ in enumerate(pts):
            dcx = pts[p1][0] - pts[p2][0]
            dcy = pts[p1][1] - pts[p2][1]
            dvx = pts[p1][2] - pts[p2][2]
            dvy = pts[p1][3] - pts[p2][3]
            
            dist_c = math.sqrt((dcx*dcx) + (dcy*dcy))
            dist_v = math.sqrt((dvx*dvx) + (dvy*dvy))
            d_c[p1,p2] = dist_c
            d_v[p1,p2] = dist_v
    return d_c, d_v, ids


# Takes in a list of nx2 points, each entry of nx2 points corresponds to 1 pedestrian's trajectory
def traj_mat(traj_arr):
    d = np.zeros((len(traj_arr),len(traj_arr)))
    # Compute DTW distance between every pair of trajectories and add to distance matrix
    for (m,n) in list((i,j) for ((i,_),(j,_)) in itertools.permutations(enumerate(traj_arr), 2)):
        dtw_dist = tdist.dtw(np.transpose(traj_arr[m]),np.transpose(traj_arr[n]))
        d[m,n] = 0.1*dtw_dist # Originally 0.2*dtw_dist
    return d

def vote_maj(adj):
    return np.sum(adj, axis=0) >= 2
def vote_unan(adj):
    return np.sum(adj, axis=0) == 3
def vote_weight(adj):
    return (0.5*adj[0] + 0.5*adj[1] + 1*adj[2]) >= 1.5

def graph_group(distances, cutoffs, ids):
    adjacency = vote_weight([dist <= co for dist, co in zip(distances, cutoffs)])
    unvisited = set([i for i in range(len(ids))])
    queue = set({})
    clusters = []
    while len(unvisited) != 0 or len(queue) != 0:
        if len(queue) == 0:
            clusters.append(set({}))
            queue.add(unvisited.pop())
        curr = queue.pop()
        clusters[-1].add(curr)
        adj, = adjacency[curr].nonzero()
        next_ids = set([i for i in adj if i in unvisited])
        queue |= next_ids
        unvisited -= next_ids
    return [set(ids[list(x)]) for x in clusters]


# Group pedestrians together
def get_groups(people):
    groups = []
    pts = []
    ppl = []
    g_index = 0
    c_arr = []
    for n, p in enumerate(people):
        length = len(p.bbox_history)
        if (length < 3):
            continue
        if length % 2 == 0:
            length = length - 1
        
        # TODO: Loosen requirement for 15 being necessary
        # For filtering, require at least 15 bboxes
#         if len(p.bbox_history) < 15:
#             continue
        bbox = p.bbox_history[-min(15, length):]
        centroids = centroid_list(np.array(bbox)[:,:4])
        # Correct the points
        for i, c in enumerate(centroids):
            centroids[i] = perspective_correction(c)
        
        # Get smoothed (filtered) centroids and velocities
        (cx, cy, vx, vy) = smooth(np.array(centroids), min(15, length), 1)
        c_list = np.array([0.05*cx,0.05*cy])
        c_arr.append(c_list)
        # Just need most recent values
        cx = cx[-1]
        cy = cy[-1]
        vx = vx[-1]
        vy = vy[-1]
        
        cx = 0.05*cx
        cy = 0.05*cy
        pt = (cx, cy, vx, vy)
        pts.append(pt)
        ppl.append(p)
    if len(pts) < 2:
        return []
#     print(c_arr)

    dist_c, dist_v, ids = dist_mats(pts)
    dist_traj = traj_mat(c_arr)
    
    print([ppl[i].person_id for i in ids])
    print("Centroid Distance: ")
    print(dist_c)
    print("Velocity Distance: ")
    print(dist_v)
    print("Trajectory Distance: ")
    print(dist_traj)
    
    li = graph_group([np.array(dist_c), np.array(dist_v), np.array(dist_traj)], [4,2,2.5], np.array(ids))
    print(li)
    
    print("-----------------------------------------------------------")
    for i in li:
        g_ppl_list = []
        if len(i) < 2:
            continue
        for v in i:
            # This needs to use ID of ppl list not pedestrian ID
            g_ppl_list.append(ppl[v])
        
        # Create new group and add both centroids to it
        groups.append([])
        groups[g_index] = Group(g_ppl_list, 0)
        g_index = g_index + 1
    
#     # Perform Mean-Shift fitting on this list
#     MS = MeanShift(max_iter=300)
#     clustering = MS.fit(np.array(pts))
#     print("Estimated bandwidth: ", estimate_bandwidth(np.array(pts)))
#     print(clustering.labels_)
#     print(clustering.cluster_centers_)
#     print(clustering.n_iter_)
    
    
#     # THE FOLLOWING GROUPING LOGIC ISN'T CORRECT!
#     d = defaultdict(list)
#     # Iterate list with enumerate.
#     for index, e in enumerate(clustering.labels_):
#         d[e].append(index)
    
# #     print(d)
    
#     for k, v in d.items():
#         # If there are not at least 2 pedestrians within a detected cluster, or the cluster is given label "-1" -> meaning they aren't within any clusters
#         if len(v) < 2 or k == -1:
#             continue
#         print(k, " has len >= 2")
#         l = []
#         for i in v:
#             # All ppl[i] here go in 1 group
#             l.append(ppl[i])

#         # Create new group and add both centroids to it
#         groups.append([])
#         groups[g_index] = Group(l, 0)
#         g_index = g_index + 1
    
    # Ensures there are no duplicate people added (this is leftover from previous approach, probably not needed)
#     for k in range(len(groups)):
#         groups[k].people = list(set(groups[k].people))
    return groups
    

# From list of n previous groups, determine what ID to give to current group
def set_group_ids(groups, prev_groups, gID_count):
    gID_list = []
            
    # if 1 person leaves group -> 
    # if 1 person joins group -> 
    # no change
    # take largest group that is sub or superset
    for g, _ in enumerate(groups):
        g_id = -1
        color = (0,0,0)
        pIDs = set([p.person_id for p in groups[g].people])
        for i in range(len(prev_groups)):
            for j in range(len(prev_groups[i])):
                pIDs_prev = set([p.person_id for p in prev_groups[i][j].people])
    #             if all(item in pIDs for item in pIDs_prev):
                if pIDs_prev.issuperset(pIDs) or pIDs.issuperset(pIDs_prev):
                    g_id = prev_groups[i][j].gID
                    color = prev_groups[i][j].color
        if g_id == -1:
            groups[g].gID = gID_count
            gID_count = gID_count + 1
        else:
            groups[g].gID = g_id
            groups[g].color = color
        gID_list.append(g_id)
    
    # Now check duplicate groups
    D = defaultdict(list)
    for i, item in enumerate(gID_list):
        D[item].append(i)
    D = {k:v for k,v in D.items() if len(v)>1}
    
    # -1 represents a new group has been made so ignore these
    if -1 in D:
        D.pop(-1)

    # For each duplicate ID found, find biggest group -> this group retains the ID, all others get set to a new Group
    for i in D:
        max_len = 0
        best_j = 0
        for j in D[i]:
            l = len(groups[j].people)
            if l > max_len:
                max_len = l
                best_j = j
        for x in D[i]:
            if x == best_j:
                continue
            # Remake any group that isn't the largest duplicate group
            # Largest group will keep previous group ID
            groups[x] = Group(groups[x].people, gID_count)
            gID_count = gID_count + 1
    return groups, gID_count


# Using specification of 7 frames per second from dataset information
def velocity(p1, p2, num_frames):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    
    # 7 fps
    mag = 7 * math.sqrt((dx*dx) + (dy*dy)) / num_frames
    angle = math.atan2(dy, dx)
    return mag, angle

def velocity_components(p1, p2, num_frames):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    
    return (dx/num_frames, dy/num_frames)

def euclid_dist(p1, p2):
    x = p1[0] - p2[0]
    y = p1[1] - p2[1]
    return math.sqrt((x*x) + (y*y))

def euclid_dist_list(p1, p2):
    dist = []
    if len(p1) != len(p2):
        return []
    for i in range(len(p1)):
        x = p1[i][0] - p2[i][0]
        y = p1[i][1] - p2[i][1]
        dist.append(math.sqrt((x*x) + (y*y)))
    return dist

# Use this function to apply transform to correct perspective shift
def perspective_correction(pt):
    pt = np.append(np.array(pt), 1).transpose()
    pt = PERSPECTIVE_TRANSFORM @ pt
    return (pt[0] / pt[2], pt[1] / pt[2])

In [ ]:
# Takes in a list of centroids (c_x, c_y) -> must be at least 15 in the list
# Outputs smoothed centroid values and velocity values (c_x, c_y, v_x, v_y)
# Returns None if pos is not sufficient length

# Best values found were window_size = 15, order = 2
def smooth(pos, window_size, order):
    if pos.shape[0] >= window_size:
        z_x = scipy.signal.savgol_filter(pos[:,0], window_size, order, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
        z_y = scipy.signal.savgol_filter(pos[:,1], window_size, order, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
        z_vx = scipy.signal.savgol_filter(pos[:,0], window_size, order, deriv=1, delta=1.0, axis=-1, mode='interp', cval=0.0)
        z_vy = scipy.signal.savgol_filter(pos[:,1], window_size, order, deriv=1, delta=1.0, axis=-1, mode='interp', cval=0.0)
        return (z_x, z_y, z_vx, z_vy)
    else:
        raise ValueError("pos must be of at shape (n,2) where n >= ", window_size, "!")
        return None

# Runtime Logic & UI

In [ ]:
height, width, channels = images[0].shape
detector = YoloDetector(width, height)
cur_people = []
cur_person_id = 0

fourcc = cv2.VideoWriter_fourcc(*'XVID')
framerate = 7
video_out = cv2.VideoWriter("tracking-output.avi", fourcc, framerate, (width,height))

# Task 1b/c, Task 2 relevant params
ENABLE_TASK_VIEW = True
ENABLE_GROUND_TRUTH_DEBUG_VIEW = True

# primarily: [TRACKING PARAMETERS]
ENABLE_TRACKING_DEBUG_VIEW = False
OVERLAP_THRESHOLD_RATIO = 0.4

DETECT_WINDOW = 'Detection'
DETECT_COLOR = (255,0,0) # blue
UNTRACK_COLOR = (255,0,191) # purple
TRACK_COLOR = (0,0,255) # red
GROUND_TRUTH_COLOR = (0,0,0) # black (170,0,255) # pink

FLOW_WINDOW = 'Flow'
MASK_WINDOW = 'Mask'

INTERACTIVE_CONTROL = True
ESC_KEY = 27
PAUSE_KEY = ord('p')
NEXT_KEY = ord('n')
KEY_LIST = [ESC_KEY, PAUSE_KEY, NEXT_KEY]

STD_DEV = 10 # px
CONST_hc = 0.3
CONST_std_m = STD_DEV
CONST_alpha_occlus = 0.6
EUCLIDEAN_THRES = 100-25 # px, for Hungarian

FRAME_START = 0 #460 #390 #128
DRAW_PARTICLES = False

# https://stackoverflow.com/questions/15933741/how-do-i-catch-a-numpy-warning-like-its-an-exception-not-just-for-testing
np.seterr(all='raise') 
is_interactive = INTERACTIVE_CONTROL

# Task 2 initialisation
cur_img_display = copy.copy(images[FRAME_START])
cur_img_display = generic_text(cur_img_display, 'Select ROI and press ENTER', (0,20))
task2_roi = cv2.selectROI(DETECT_WINDOW, cur_img_display)
task2_ids = set() # IDs inside the Task 2 ROI

# Store previous 3 groups
num_prev_frames = 1
prev_groups = [None] * num_prev_frames

gID_count = 0

for n, cur_img in enumerate(images[FRAME_START:]):

    ###############################
    # [DETECTION & BACKGROUND SUBTRACTION]
    # setup
    cur_img_gray = cv2.cvtColor(cur_img,cv2.COLOR_BGR2GRAY)
    bw_mask = bg_subtract(cur_img)
    
    # 1. apply detector on new frame
    detected_boxes = detector.get_detection(cur_img)
    
    ###############################
    # [TRACKING]
    # 2. match detections to existing Person
    if n > 0:
        __, __, flow_rgb, flow_res = get_optical_flow(prev_img_gray, cur_img_gray)
        # associate trackers (existing Person) to detected bboxes by distance
        p_idx_list, d_idx_list, unp_idx_list, und_idx_list = hungarian_algorithm(cur_people, detected_boxes, EUCLIDEAN_THRES)
        
        # paired (trackers, detected)
        for index_pair in range(len(p_idx_list)):
            # update tracker to use detector's coords
            p = cur_people[ p_idx_list[index_pair] ]
            cur_x, cur_y, cur_w, cur_h = detected_boxes[ d_idx_list[index_pair] ]
            # tracker prediction (results unused...)
            cur_vx, cur_vy, cur_std = get_velocity_stats(p, flow_res, bw_mask)
            p.predict_tracker(cur_vx, cur_vy, STD_DEV, cur_img, n)
            # tracker update
            detected_bbox = (cur_x, cur_y, cur_w, cur_h, n, False)
            p.update_tracker(detected_bbox, cur_img, bw_mask, CONST_hc, None, None, False)
            # NOTE: our detector is likely more accurate than our tracker; atm I don't know
            # how to directly associate the detected bbox to the tracker with the
            # color histogram approach?
            
        # unpaired trackers
        for unpaired_index in unp_idx_list:  # (no index pairing required)
            # predict
            p = cur_people[unpaired_index]
            cur_vx, cur_vy, cur_std = get_velocity_stats(p, flow_res, bw_mask)
            predicted_bbox = p.predict_tracker(cur_vx, cur_vy, STD_DEV, cur_img, n)
            predicted_bbox = tuple(list(predicted_bbox) + [n, True])
            # update
            p.update_tracker(predicted_bbox, cur_img, bw_mask, CONST_hc, 
                             CONST_std_m, CONST_alpha_occlus, True)
            
    else:
        # unpaired detected bboxes (see Step 3.)
        und_idx_list = list(range(len(detected_boxes))) # all undetected
        p_idx_list = [] # to aid plotting
        d_idx_list = []
        unp_idx_list = []

    # 3. create new Person for each unpaired bbox
    unpaired_start_index = len(cur_people)
    for unpaired_index in und_idx_list:  # (no index pairing required)
        cur_x, cur_y, cur_w, cur_h = detected_boxes[ unpaired_index ]
        p = Person( (cur_x,cur_y,cur_w,cur_h,n,False), width,height,100, 
                   cur_person_id, cur_img, bw_mask, CONST_hc)
        cur_people.append(p)
        cur_person_id += 1
    
    # NOTE: we remove stale trackers after plotting results, for coding convenience
    
    ###############################
    # [PLOT TRACKING-ONLY RESULTS]
    cur_img_display = copy.copy(cur_img)
    if ENABLE_TRACKING_DEBUG_VIEW:
        post_x, post_y, post_w, post_h = Person.INNER_BORDER_RECT
        cur_img_display = cv2.rectangle(cur_img_display, (post_x, post_y), 
                                        (post_x+post_w, post_y+post_h), (0,0,0), 1)
        cur_img_display = cv2.rectangle(cur_img_display, (Person.BORDER_PX, Person.BORDER_PX), 
                                        (width-Person.BORDER_PX, height-Person.BORDER_PX), (0,0,0), 1)
#         cv2.putText(cur_img_display, f'Fm {FRAME_START+n}', (0,20), FONT, 0.75, (255,0,0),2)  # frame id
    
    # paired (trackers, detected)
    for index_pair in range(len(p_idx_list)):
        p = cur_people[ p_idx_list[index_pair] ]
        cur_bbox = p.get_latest_ords()[:4]
        #cur_bbox = detected_boxes[ d_idx_list[index_pair] ]
        cur_img_display = draw_rectangle(cur_img_display, cur_bbox, str(p.person_id), FONT, DETECT_COLOR)
    
    # unpaired trackers
    for unpaired_index in unp_idx_list:
        p = cur_people[unpaired_index]
        cur_bbox = p.get_latest_ords()[:4]
        cur_img_display = draw_rectangle(cur_img_display, cur_bbox, str(p.person_id), FONT, TRACK_COLOR)
    
    # unpaired detected bboxes
    for unpaired_index in range(unpaired_start_index, len(cur_people)):
        p = cur_people[unpaired_index]
        cur_bbox = p.get_latest_ords()[:4]
        cur_img_display = draw_rectangle(cur_img_display, cur_bbox, str(p.person_id), FONT, UNTRACK_COLOR)
        
    # tracker predictions, particles
    for p in cur_people:
        if p.last_prediction == None: # i.e ignore new Person
            continue
        cur_img_display = p.draw(cur_img_display, TRACK_COLOR, 
                                 enable_particles=DRAW_PARTICLES, draw_last_predicted=True)
    
    ###############################
    # [TRACKING CLEAN-UP]
    # remove stale trackers
    p_index = 0
    while p_index < len(cur_people):
        p = cur_people[p_index]
        if p.is_stale_tracker(cur_img):
            del cur_people[p_index]
            continue
        p_index += 1
    
#     ###############################
    # [TASK 2 LOGIC]
    task2_entered = 0
    task2_left = 0
    for p in cur_people: # probably a non-optimal extra loop
        cur_bbox = p.get_latest_ords()[:4]
        # 1. check for overlapping area (must be more than just a boundary touch)
        # 2. check for existing membership
        if area_overlap(cur_bbox, task2_roi) > 0:
            # entered box
            if p.person_id not in task2_ids:
                task2_entered += 1
                task2_ids.add(p.person_id)
        else: # no overlap
            # left box
            if p.person_id in task2_ids:
                task2_left += 1
                task2_ids.remove(p.person_id)

    ###############################
    # [TASK 3 LOGIC]
    groups = []
    print(n)
    groups = get_groups(cur_people)
    groups, gID_count = set_group_ids(groups, prev_groups, gID_count)

    # Print centroid ID in white
    for p in cur_people:
        p.color = (255,255,255)
#         org = centroid(p.get_latest_ords()[:4])
#         cv2.putText(cur_img_display, str(p.person_id), org, fontFace=cv2.FONT_HERSHEY_SIMPLEX, color=(255,255,255), fontScale=1, thickness=3)
    
    num_grouped_pedestrians = 0
    # Color coordinate groups
    for k in range(len(groups)):
        num_grouped_pedestrians = num_grouped_pedestrians + len(groups[k].people)
        for l in range(len(groups[k].people)):
            groups[k].people[l].color = groups[k].color
#             org = centroid(groups[k].people[l].get_latest_ords()[:4])
#             cv2.putText(cur_img_display, str(groups[k].people[l].person_id), org, fontFace=cv2.FONT_HERSHEY_SIMPLEX, color=groups[k].color, fontScale=1, thickness=3)
    
    ###############################
    # [GROUND TRUTH]
    
    
    ###############################
    # [PLOT FINALISED RESULTS]
    # ** display user-selected bounding box (for Task 2)
    cur_img_display = cv2.rectangle(cur_img_display, task2_roi[:2], 
                                    (task2_roi[0]+task2_roi[2], task2_roi[1]+task2_roi[3]), 
                                    (0,0,0), 1)
    
    if ENABLE_GROUND_TRUTH_DEBUG_VIEW:
        cur_gt_dict = ground_truth[FRAME_START+n]
        for cur_gt_dict_key in cur_gt_dict:
            cur_bbox = cur_gt_dict[cur_gt_dict_key]
            cur_bbox = tuple(map(int, cur_bbox))
            cur_img_display = draw_rectangle(cur_img_display, cur_bbox, None, FONT, GROUND_TRUTH_COLOR, thickness=1)
    
    if ENABLE_TASK_VIEW:
        # draw trajectories
        for p in cur_people:
            cur_img_display = p.draw_trajectory(cur_img_display)
        # draw bboxes and labels on top
        for p in cur_people:
            cur_img_display = p.draw(cur_img_display)        

    # [DISPLAY STATS]
    # ** frame id
    cv2.putText(cur_img_display, f'Fm {FRAME_START+n}', (0,20), FONT, 0.75, (255,0,0),2)
    # ** Task 1c: pedestrians seen since frame 1
    # ** Task 2
    text_display = [
        f'T1c: Total Pedestrns: {cur_person_id}',
        f'T2a: Cur ROI Entries: {task2_entered}',
        f'T2b: Cur ROI Depart: {task2_left}',
        f'T2 : Cur ROI Inside : {len(task2_ids)}',
        f'T3a : Pedestrns Alone: {len(cur_people) - num_grouped_pedestrians}',
        f'T3a : Pedestrns Group: {num_grouped_pedestrians}'
    ]
    for cur_text_index, cur_text in enumerate(text_display): 
        cur_img_display = generic_text(cur_img_display, cur_text, (0,height-(len(text_display)-cur_text_index)*22))
    
    ###############################
    # [ENABLE WINDOWS]
    cv2.imshow(DETECT_WINDOW, cur_img_display)
    if ENABLE_TRACKING_DEBUG_VIEW:
        if n > 0:
            flow_arrow = draw_flow(cur_img_gray, flow_res, step=8)
            cv2.imshow(FLOW_WINDOW, flow_arrow)
        else:
            cv2.imshow(FLOW_WINDOW, cur_img_display)
        cv2.imshow(MASK_WINDOW, cv2.bitwise_and(cur_img, cur_img, mask=bw_mask))

    ###############################
    # [BOOK-KEEPING FOR NEXT ITERATION]
    # save results
    video_out.write(cur_img_display)
    
    # setup for next iteration
    prev_img_gray = cur_img_gray
    prev_groups[n%num_prev_frames] = groups
    
    ###############################
    # [PLAYBACK CONTROL]
    if not is_interactive:
        ch = cv2.waitKey(5) # millis
    else:
        while True:
            ch = cv2.waitKey(0)
            if ch in KEY_LIST:
                break
    if ch == ESC_KEY: # press ESC on the main window to exit
        cv2.destroyAllWindows()
        break
    elif ch == PAUSE_KEY:
        is_interactive = not is_interactive
    elif ch == NEXT_KEY:
        pass # continue; do nothing

# close results
video_out.release()
print('Video result saved')